In [1]:
import numpy as np
from sklearn import datasets
from sklearn import preprocessing
from sklearn import model_selection

In [2]:
features, targets = datasets.load_breast_cancer(return_X_y=True)

In [3]:
features.shape

(569, 30)

In [4]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(features, targets, test_size=0.2, random_state=0)

In [5]:
n_features = x_train.shape[-1]
n_features

30

In [6]:
# Input features are centered and scaled to unit variance:
feature_encoder = preprocessing.StandardScaler()
x_train = feature_encoder.fit_transform(x_train)
x_test = feature_encoder.transform(x_test)

In [7]:
x_train.shape[0], x_test.shape[0]

(455, 114)

In [8]:
# number of neurons per layer, the last element must be 1
n_neurons = np.array([100, 10, 1])
n_branches = 20  # number of dendritic brancher per neuron

In [9]:
n_neurons

array([100,  10,   1])

---

In [10]:
import torch.nn as nn
import torch

In [11]:
class DentricLayer(nn.Module):
    def __init__(self, 
                 n_input:int,
                 n_output:int, 
                 n_branches:int,
                 hyperplane_bias_magnitude:float = 1.):
        
        super(DentricLayer, self).__init__()
        
        # self.n_input = n_input
        # self.n_branch = n_branch
        # self.n_output = n_output
        
        self.dgn_weights = torch.zeros(n_output, n_branches, n_input + 1)
        self.dgn_hyperplanes = torch.normal(0, 1, [n_output, n_branches, n_input + 1])
        self.dgn_hyperplanes = self.dgn_hyperplanes / torch.linalg.norm(self.dgn_hyperplanes[:, :, 1:], axis=(1, 2))[:, None, None]
        self.hyperplane_bias_magnitude = hyperplane_bias_magnitude
        
    def forward(self, X):
        r_in = torch.hstack([X, torch.ones((X.shape[0], 1))])
        side_info = torch.hstack([X, torch.ones((X.shape[0], 1)) * self.hyperplane_bias_magnitude])
        h_dot_side_indo = torch.dot(self.dgn_hyperplanes, side_info)
        gate_values = torch.heaviside(h_dot_side_indo, 0)
        effective_weights  = torch.dot(gate_values, self.dgn_weights).sum(dim=1)
        r_out = torch.dot(effective_weights, r_in)
        
        return r_out

In [12]:
h = torch.normal(0, 1, [100, 20, 30 + 1])
h = h / torch.linalg.norm(h[:, :, :-1], axis=(1, 2))[:, None, None]

In [13]:
x = np.hstack([np.ones((x_train.shape[0], 1)), x_train])
x = torch.FloatTensor(x)

In [16]:
h.shape, x.shape

(torch.Size([10, 20, 31]), torch.Size([455, 31]))

In [17]:
tensor_dot = torch.tensordot(h, x, dims=([2], [1]))

In [18]:
tensor_dot.shape

torch.Size([10, 20, 455])

In [19]:
a = torch.randn(3, 5, 4, 6)
b = torch.randn(6, 4, 5, 3)
torch.tensordot(a, b, dims=([2, 1, 3], [1, 2, 0])) # 4, 5, 6 | 4, 5, 6

tensor([[19.1452,  3.2551,  0.8042],
        [-8.6342, 10.4805,  1.9739],
        [-3.4690, 10.5930,  0.7933]])

In [20]:
gate = torch.heaviside(tensor_dot, torch.tensor(0.))

In [21]:
dgn_weights = torch.zeros(10, 20, 30 + 1)

In [22]:
dgn_weights.shape, gate.shape

(torch.Size([10, 20, 31]), torch.Size([10, 20, 455]))

In [30]:
effective_weights = torch.tensordot(gate, dgn_weights, dims=([0], [0])).sum(dim=1)

In [31]:
effective_weights.shape

torch.Size([20, 20, 31])

In [25]:
x.shape, effective_weights.shape

(torch.Size([455, 31]), torch.Size([10, 10, 31]))

In [26]:
effective_weights[:None] * x

RuntimeError: The size of tensor a (10) must match the size of tensor b (455) at non-singleton dimension 1

In [27]:
torch.dot(effective_weights[:, None], x)

RuntimeError: 1D tensors expected, but got 4D and 2D tensors

In [ ]:
effective_weights